# Movie Recommendation - using cosine similarity
> Using Count Vectoriser and Cosine Similarity.
- toc: false 
- badges: true
- comments: false
- categories: [jupyter, sklearn, CountVectorizer, cosine_similarity]
- author: Venkataramani, Suja

### Overview  

We will write a simple movie recommendation system using IMDB dataset We will pre-process the text columns by extracting key works using RAKE (Rapid Automatic Keyword Extraction), remove spaces and commas. Then we will convert it into numeric matrix using both Count Vectorizer and TF-IDF Vectorizer and calculate cosine similarity. We will try find 10 movies which match a given movie.

### Method  

Download top 250 movie ratings from [Data World](https://data.world/studentoflife/imdb-top-250-lists-and-5000-or-so-data-records).

We will need Rake to use some key methods in nltk, so install that using "pip install rake-nltk" on a command prompt.

In [21]:
# Import required packages.
from rake_nltk import Rake
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [22]:
# Show all columns.
pd.options.display.max_columns = None

In [23]:
# Read the data into a dataframe.
data = pd.read_csv("Data/IMDB_Top250movies2_OMDB_Detailed.csv")

In [24]:
# Show top 5 rows.
data.head(5)

,Unnamed: 0,Title,Year,Rated,Released,Runtime,Genre,Director,Writer,Actors,Plot,Language,Country,Awards,Poster,Ratings.Source,Ratings.Value,Metascore,imdbRating,imdbVotes,imdbID,Type,tomatoMeter,tomatoImage,tomatoRating,tomatoReviews,tomatoFresh,tomatoRotten,tomatoConsensus,tomatoUserMeter,tomatoUserRating,tomatoUserReviews,tomatoURL,DVD,BoxOffice,Production,Website,Response
0,1,The Shawshank Redemption,1994,R,14 Oct 1994,142 min,"Crime, Drama",Frank Darabont,"Stephen King (short story ""Rita Hayworth and S...","Tim Robbins, Morgan Freeman, Bob Gunton, Willi...",Two imprisoned men bond over a number of years...,English,USA,Nominated for 7 Oscars. Another 19 wins & 30 n...,https://images-na.ssl-images-amazon.com/images...,Internet Movie Database,9.3/10,80.0,9.3,"1,825,626",tt0111161,movie,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://www.rottentomatoes.com/m/shawshank_rede...,27 Jan 1998,NaN,Columbia Pictures,NaN,True
1,2,The Godfather,1972,R,24 Mar 1972,175 min,"Crime, Drama",Francis Ford Coppola,"Mario Puzo (screenplay), Francis Ford Coppola ...","Marlon Brando, Al Pacino, James Caan, Richard ...",The aging patriarch of an organized crime dyna...,"English, Italian, Latin",USA,Won 3 Oscars. Another 23 wins & 27 nominations.,https://images-na.ssl-images-amazon.com/images...,Internet Movie Database,9.2/10,100.0,9.2,"1,243,444",tt0068646,movie,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://www.rottentomatoes.com/m/godfather/,09 Oct 2001,NaN,Paramount Pictures,http://www.thegodfather.com,True
2,3,The Godfather: Part II,1974,R,20 Dec 1974,202 min,"Crime, Drama",Francis Ford Coppola,"Francis Ford Coppola (screenplay), Mario Puzo ...","Al Pacino, Robert Duvall, Diane Keaton, Robert...",The early life and career of Vito Corleone in ...,"English, Italian, Spanish, Latin, Sicilian",USA,Won 6 Oscars. Another 10 wins & 20 nominations.,https://images-na.ssl-images-amazon.com/images...,Internet Movie Database,9.0/10,85.0,9.0,"856,870",tt0071562,movie,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://www.rottentomatoes.com/m/godfather_part...,24 May 2005,NaN,Paramount Pictures,http://www.thegodfather.com/,True
3,4,The Dark Knight,2008,PG-13,18 Jul 2008,152 min,"Action, Crime, Drama",Christopher Nolan,"Jonathan Nolan (screenplay), Christopher Nolan...","Christian Bale, Heath Ledger, Aaron Eckhart, M...",When the menace known as the Joker emerges fro...,"English, Mandarin","USA, UK",Won 2 Oscars. Another 151 wins & 153 nominations.,https://images-na.ssl-images-amazon.com/images...,Internet Movie Database,9.0/10,82.0,9.0,"1,802,351",tt0468569,movie,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://www.rottentomatoes.com/m/the_dark_knight/,09 Dec 2008,"$533,316,061",Warner Bros. Pictures/Legendary,http://thedarkknight.warnerbros.com/,True
4,5,12 Angry Men,1957,APPROVED,01 Apr 1957,96 min,"Crime, Drama",Sidney Lumet,"Reginald Rose (story), Reginald Rose (screenplay)","Martin Balsam, John Fiedler, Lee J. Cobb, E.G....",A jury holdout attempts to prevent a miscarria...,English,USA,Nominated for 3 Oscars. Another 16 wins & 8 no...,https://images-na.ssl-images-amazon.com/images...,Internet Movie Database,8.9/10,96.0,8.9,"494,215",tt0050083,movie,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://www.rottentomatoes.com/m/1000013-12_ang...,06 Mar 2001,NaN,Criterion Collection,http://www.criterion.com/films/27871-12-angry-men,True


### RAKE - Rapid Automatic Keyword Extraction

Rake is a language-independent unsupervised algorithm for extracting keywords from documents. It is not domain dependent and does not depend on arbitrary selection of n-grams to use as a sliding window. Instead it uses the stopwords and phrase delimiters to break a document into keywords. 

The words are then given scores:

freq(w): number of times the word occurs in a document  
deg(w): sum of the number of words the word co-occurs in the document  
score(w): deg(w)/freq(w)   


[RAKE Research Paper](https://www.researchgate.net/publication/227988510_Automatic_Keyword_Extraction_from_Individual_Documents)  
[MonkeyLearn](https://monkeylearn.com/keyword-extraction/)

In [25]:
data["key_words"] = ""
r = Rake()
for index, row in data.iterrows():
    r.extract_keywords_from_text(row['Plot'])   
    key_word_scores = r.get_word_degrees() 
    data.at[index, "key_words"] = list(key_word_scores.keys()) 

In [27]:
data["key_words"][0:5]

0    [finding, solace, common, decency, years, two,...
1    [reluctant, son, clandestine, empire, aging, p...
2    [early, life, tightens, portrayed, grip, famil...
3    [menace, known, wreaks, havoc, chaos, people, ...
4    [forcing, colleagues, evidence, justice, preve...
Name: key_words, dtype: object

In [28]:
# Pre-process by splitting by comma to get the words.
data["Genre"] = data["Genre"].map(lambda x: x.split(","))
data["Actors"] = data["Actors"].map(lambda x: x.split(","))
data["Director"] = data["Director"].map(lambda x: x.split(","))

In [29]:
# For each of the list of words, remove spaces.
for index, row in data.iterrows():
    data.at[index, "Genre"] = list(map(lambda x: x.replace(" ", ""), row["Genre"]))
    data.at[index, "Actors"] = list(map(lambda x: x.replace(" ", ""), row["Actors"]))
    data.at[index, "Director"] = list(map(lambda x: x.replace(" ", ""), row["Director"]))

In [30]:
# Create a new attribute to store bag of words to feed.
data["bag_of_words"] = ""
for index, row in data.iterrows():
    # Create an appended list from Genre, Actors, Directore and keywords.
    # The create a string with these, delimited by space.
    data.at[index, "bag_of_words"] = "".join(item + " " for item in row["Genre"] + row["Actors"] + row["Director"] + row["key_words"])

In [31]:
data["bag_of_words"][0:5]

0    Crime Drama TimRobbins MorganFreeman BobGunton...
1    Crime Drama MarlonBrando AlPacino JamesCaan Ri...
2    Crime Drama AlPacino RobertDuvall DianeKeaton ...
3    Action Crime Drama ChristianBale HeathLedger A...
4    Crime Drama MartinBalsam JohnFiedler LeeJ.Cobb...
Name: bag_of_words, dtype: object

### CountVectorizer  

Converts a collection of documents into matrix of number of documents (rows) and number of features (columns) with the count of the words in each cell. sklearn returns this a a sparse matrix - which is an efficient way of storing data when the the matrix is mostly zeroes. The number of features is the sum of all the words in the documents.

### TfIdfVectorizer  

This calculated as:  
tf-idf(t, d) = tf(t, d) * idf(t)  
idf(t) = log(n/(df(t)) + 1)  

where:  
    n = total number of documents  
    tf(t, d) = term frequency = number of times term occurs in the document/all the terms in the document  
    df(t) = document frequecy = number of document in which a document occurs/total number of documents  

### Cosine Similarity  

It is the angle between two vectors in multidimentional space. The intuition behind finding cosine similarlity between two documents over Euclidean distance is that the size two documents are not indicative of their similarity. If each of the word is a dimension of a document vector, then we would like to find another document which points in the same direction regardless of the number of times a word occurs in any document.

[MachineLearningPlus](https://www.machinelearningplus.com/nlp/cosine-similarity/)


In [32]:
cv = CountVectorizer()
# Creates a sprarse matrixof number documents vs number of features and count.
cv_matrix = cv.fit_transform(data["bag_of_words"])
cos_sim =  cosine_similarity(cv_matrix, cv_matrix)

In [33]:
# 250 rows (movie rating rows) vs 3183 columns (features).
cv_matrix

<250x3183 sparse matrix of type '<class 'numpy.int64'>'
	with 5551 stored elements in Compressed Sparse Row format>

In [34]:
# Convert sparce matrix to dense matrix.
d_matrix = cv_matrix.todense()
df = pd.DataFrame(d_matrix,
                columns=cv.get_feature_names())
df                

0   
1             0            0              0          0               0   
2             0            0              0          0               0   
3             0            0              0          0               0   
4             0            0              0          0               0   
..          ...          ...            ...        ...             ...   
245           0            0              0          0               0   
246           0            1              0          0               0   
247           0            0              0          0               0   
248           0            0              0          0               0   
249           0            0              0          0               0   

     vienna  vietnam  view  viking  village  vincentcassel  vincentd  \
0         0        0     0       0        0              0         0   
1         0        0     0       0        0              0         0   
2         0        0     0       0        0              0         0   
3         0        0     0       0        0              0         0   
4         0        0     0       0        0              0         0   
..      ...      ...   ...     ...      ...            ...       ...   
245       0        0     0       0        0              0         0   
246       0        0     0       0        0              0         0   
247       0        0     0       0        0              0         0   
248       0        0     0       0        0              0         0   
249       0        0     0       0        0              0         0   

     vinniejones  violadavis  violence  violent  virginiacherrill  virus  \
0              0           0         0        0                 0      0   
1              0           0         0        0                 0      0   
2              0           0         0        0                 0      0   
3              0           0         0        0                 0      0   
4              0           0         0        0                 0      0   
..           ...         ...       ...      ...               ...    ...   
245            0           0         0        0                 0      0   
246            0           0         0        0                 0      0   
247            0           0         0        0                 0      0   
248            0           0         0        0                 0      0   
249            0           0         0        0                 0      0   

     visions  visit  vito  vittoriodesica  vivicaa  vivienleigh  \
0          0      0     0               0        0            0   
1          0      0     0               0        0            0   
2          0      0     1               0        0            0   
3          0      0     0               0        0            0   
4          0      0     0               0        0            0   
..       ...    ...   ...             ...      ...          ...   
245        0      0     0               0        0            0   
246        0      0     0               0        0            0   
247        0      0     0               0        0            0   
248        0      0     0               0        0            0   
249        0      0     0               0        0            0   

     vladasbagdonas  voldemort  volunteers  véraclouzot  wai  waifish  \
0                 0          0           0            0    0        0   
1                 0          0           0            0    0        0   
2                 0          0           0            0    0        0   
3                 0          0           0            0    0        0   
4                 0          0           0            0    0        0   
..              ...        ...         ...          ...  ...      ...   
245               0          0           0            0    0        0   
246               0          0           0            0    0        0   
247           

In [35]:
# Create a series with Title, index will be auto-created which can be used to locate titles.
ind = pd.Series(data["Title"])

In [36]:
# Function to find movie recomendations.
def recommend(title, cos_sim = cos_sim):
    recommended_movies = []
    # Find the index of the movie given.
    movie_ind = ind[ind==title].index[0]
    # Find the row in cosine similarity for the movie and sort the values in descending order.
    series_score = pd.Series(cos_sim[movie_ind]).sort_values(ascending=False)
    # index 0 is the movie itself, so get the next 10 movies indexes.
    top_10 = list(series_score[1:11].index)

    for i in top_10:
        # Now find the tile for the 10 indexes.
        movie = list(data["Title"])[i]
        recommended_movies.append(movie)

    return recommended_movies

In [37]:
recommend("The Godfather")

['The Godfather: Part II',
 'Rashomon',
 'The 400 Blows',
 'Drishyam',
 'Scarface',
 'Fargo',
 'On the Waterfront',
 'Goodfellas',
 'Baby Driver',
 'Cool Hand Luke']

In [43]:
tv = TfidfVectorizer()
# Creates a sprarse matrixof number documents vs number of features and count.
tv_matrix = tv.fit_transform(data["bag_of_words"])
cos_sim =  cosine_similarity(tv_matrix, tv_matrix)

In [44]:
# Function to find movie recomendations.
def recommend(title, cos_sim = cos_sim):
    recommended_movies = []
    # Find the index of the movie given.
    movie_ind = ind[ind==title].index[0]
    # Find the row in cosine similarity for the movie and sort the values in descending order.
    series_score = pd.Series(cos_sim[movie_ind]).sort_values(ascending=False)
    # index 0 is the movie itself, so get the next 10 movies indexes.
    top_10 = list(series_score[1:11].index)

    for i in top_10:
        # Now find the tile for the 10 indexes.
        movie = list(data["Title"])[i]
        recommended_movies.append(movie)

    return recommended_movies

In [45]:
recommend("The Godfather")

['The Godfather: Part II',
 'Apocalypse Now',
 'On the Waterfront',
 'Scarface',
 'Heat',
 'Casino',
 'Wild Strawberries',
 'Rashomon',
 'All About Eve',
 'The 400 Blows']

### Conclusion  

We tried Count Vectorizer and Tf-Idf Vectorizer and different recommendations.